In [13]:
############################################################
"""
    Importing initial Notebooks and Libraries
"""
############################################################

# Jupyter notebook imports
%run twitter_sentiment.ipynb

# Library imports
import os
import pandas as pd
from datetime import date, timedelta
import alpaca_trade_api as tradeapi
from talib import RSI, OBV, ATR, STDDEV

# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

Twitter Authentication Verified


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/Kris/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
############################################################
"""
    API Authentications for Alpaca
"""
############################################################
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "v2")

In [15]:
def get_indicators(symbol):   

    indicator_scores = []
    column_names = ['symbol']
    indicator_scores = pd.DataFrame(columns = column_names)
    
    # Set the symbol row and column
    indicator_scores.loc[0, 'symbol'] = symbol
    
    # Set timeframe to 15 Minutes
    timeframe = "15Min"
    
    # Set end date to now for latest data
    end_date = pd.Timestamp.now(tz="America/New_York").isoformat()
    
    # set limit to past 50 15-minute intervals (equal to one trading day of 6.5 hours)
    limit = 50
        
    #fetch ticker data 
    ticker_data = api.get_barset(
        ticker,
        timeframe,
        limit,
        end=end_date
        )[ticker]._raw
    ticker_data_df = pd.DataFrame(data=ticker_data)
    
    ############################################################
    """ 
        Volatility - Average True Range
        Finds volatile stocks using ATR.
        Looks for stocks that typically move more than 5% per day, based on a 50 time period average, 
        
        Relative strength suggests continued outperformance while relative weakness suggests continued underperformance.
        High RSI (usually above 70) may indicate a stock is overbought, therefore it is a sell signal. 
        Low RSI (usually below 30) indicates stock is oversold, which means a buy signal. 
        
        On-balance volume (OBV) is a technical trading momentum indicator that 
        uses volume flow to predict changes in stock price.
        
        When both price and OBV are making higher peaks and higher troughs, 
        the upward trend is likely to continue.
    """
    ############################################################
    
    #fetch Average True Range(ATR) volatility indicator
    #average_true_range = pd.DataFrame(ATR(ticker_data_df['h'], ticker_data_df['l'], ticker_data_df['c'], timeperiod=14))

    #fetch Standard Deviation volatility indicator
    #std_dev = pd.DataFrame(STDDEV(ticker_data_df['c'], timeperiod=5, nbdev=1))
    std_dev = pd.DataFrame(ticker_data_df['c']).std()
    std_dev = std_dev.mean()
                           
    #Fetch RSI score
    rsi_score = pd.DataFrame(RSI(ticker_data_df['c'], timeperiod=14))
    rsi_score = rsi_score.mean()
     
    #Fetch On Balance Volume(OBV)
    obv_score = pd.DataFrame(OBV(ticker_data_df['c'], ticker_data_df['v']))
    
    if ((rsi_score <= 45) | (std_dev > 0.008554)).all(): 
    
        # if all conditions met, appends scores to dataframe
        indicator_scores.loc[0, 'close'] = ticker_data_df.loc[(len(ticker_data_df)-1), 'c']
        indicator_scores.loc[0, 'std_dev'] = std_dev #.loc[0]                     
        #indicator_scores.loc[0, 'avg_true_range'] = average_true_range.loc[(len(average_true_range)-1), 0]
        indicator_scores.loc[0, 'relative_strength_index'] = rsi_score.loc[0] #.loc[(len(rsi_score)-1), 0]
        indicator_scores.loc[0, 'on_balance_volume'] = obv_score.loc[(len(obv_score)-1), 0]
        
    else:
        pass
    
    indicator_scores.dropna(inplace=True)
    
    return indicator_scores

In [16]:
############################################################
"""
    Function that fetches the Twitter Sentiment score from twitter_sentiment.ipynb
"""
############################################################

def fetch_twitter_sentiment(ticker, search_word):
    score = compound_twitter_sentiment(ticker, search_word)
    return score

############################################################
"""
    Function scrapes Wikipedia and pulls stock data for every ticker symbol on the S&P500
"""
############################################################

# scrapes the wikipedia page relating to the S&P 500 and returns a list of DataFrame objects
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# Since we are only interested in the current list of stocks in the S&P 500, we only need the DataFrame object at index 0
sp500_list_df = table[0]
sp500_tickers = sp500_list_df[["Symbol", "Security"]]

############################################################
"""
    Stock Screener for Day Trading
    Start Main Function
"""
############################################################

# Initialize the dataframe and set the columns
indicators_df = []
column_names = ['symbol', 'close', 'std_dev', 'relative_strength_index', 'on_balance_volume']
indicators_df = pd.DataFrame(columns = column_names)

# for loop iterates through all tickers in S&P 500
for tickers in range(len(sp500_tickers["Symbol"])):
    
    # set the current ticker symbol and company security name
    company = sp500_tickers.loc[[tickers][0]]
    ticker = company.Symbol
    security = company.Security
    
    # Set timeframe to 1 Day to capture daily Volume total
    timeframe = "1D"
    
    # Set the limit of bars to just the previous time period
    limit = 1

    # Set end date to now for latest data (if run before market open to fetch yesterdays close)
    end_date = pd.Timestamp.now(tz="America/New_York").isoformat()
    
    # Initial pass of ticker symbols
    # fetch last daily barset object for ticker and put raw data into dataframe
    bar_set = api.get_barset(
        ticker,
        timeframe,
        limit,
        end=end_date
        )[ticker]._raw
    initial_check = pd.DataFrame(data=bar_set)
    
    ############################################################
    """
    # Since Day traders generally look for stocks that have at least 1 million shares traded daily, 
    # this checks to see if the daily volume for current ticker >= 1Million.
    # This also checks the last closing price of a ticker to see if it is below $1 or above $100.
    # If ticker volume < 1M or it's last closing price was above $100, 
    # we stop this iteration and continue to the next iteration of the loop.
    """
    ############################################################
           
    if initial_check.loc[0, 'v'] <= 1000000 or initial_check.loc[0, 'c'] > 100 or initial_check.loc[0, 'c'] < 1:
        continue
    else:
        # If the ticker passes initial checks, it continues to calculating the additional indicators
        
        indicators_df = indicators_df.append(get_indicators(ticker), ignore_index=True)
        
        
        ############################################################
        """ 
            Twitter and Reddit Compound Sentiment
        """
        ############################################################
        
        
        #fetches the Compound Sentiment score for the last 15 minutes of Tweets on Twitter
        twitter_comp_sentiment_score = fetch_twitter_sentiment(ticker, security)
        indicators_df.loc[tickers, 'twitter_sentiment'] = twitter_comp_sentiment_score
            
        #fetch reddit sentiment
        #reddit_sentiment = fetch_reddit_sentiment(ticker)
        #sp500_ticker_data.loc[x, 'reddit_sentiment'] = reddit_sentiment


indicators_df.set_index('symbol', inplace=True)
indicators_df

Rate limit reached. Sleeping for: 221
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/15Min 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
Rate limit reached. Sleeping for: 786


close    std_dev  relative_strength_index  on_balance_volume  \
symbol                                                                  
ATVI    93.760  39.934546                49.467030            33346.0   
NaN        NaN        NaN                      NaN                NaN   
AMD      0.000  33.719908                57.913438           -12628.0   
NaN        NaN        NaN                      NaN                NaN   
AES     26.900   8.790533                49.889057           102220.0   
...        ...        ...                      ...                ...   
NaN        NaN        NaN                      NaN                NaN   
XRX     25.680  13.090117                51.314302            -5500.0   
NaN        NaN        NaN                      NaN                NaN   
ZION    54.145  27.390632                51.668004             -967.0   
NaN        NaN        NaN                      NaN                NaN   

        twitter_sentiment  
symbol                     
ATVI                  NaN  
NaN                0.8750  
AMD                   NaN  
NaN               -0.1877  
AES                   NaN  
...                   ...  
NaN                0.2732  
XRX                   NaN  
NaN                0.2382  
ZION                  NaN  
NaN                0.0000  

[448 rows x 5 columns]

In [ ]:
############################################################
""" 

    Adds up scores of each indicator and returns a sorted list with Top 5 stock tickers
    
"""
############################################################


# adds up sentiment indicator column scores and calculates the total_indi_score
#for symbol in range(len(indicators_df)):
    #indicators_df[symbol, 'total_sentiment_score'] = float(indicators_df[rows, 'twitter_sentiment'] + indicators_df[rows, 'reddit_sentiment'])
    
# sorts indidcator_df by total_indie_score
#indicators_df.sort_by(by='total_trading_score', ascending=True)

# saves top 5 tickers into top5_stocks_df["tickers"]
#top5_stocks_df = indicators_df[:4]

# return top 5 stock recommendations
#top5_stocks_df

In [ ]:
        ############################################################
        """         
        
        # Set timeframe to 15 Minutes
        timeframe = "15Min"
    
        #fetch ticker data 
        ticker_data = api.get_barset(
            ticker,
            timeframe,
            limit,
            end=end_date
            )
    
        # ticker_data inherits dict and bars inherits list. You can iterate over them accordingly: 
        for symbols in ticker_data:
            bars = ticker_data[symbols]
            for bar in bars:
                sp500_ticker_data.loc[symbols, 'time'] = bar.t
                sp500_ticker_data.loc[symbols, 'open'] = bar.o
                sp500_ticker_data.loc[symbols, 'low'] = bar.l
                sp500_ticker_data.loc[symbols, 'high'] = bar.h
                sp500_ticker_data.loc[symbols, 'close'] = bar.c
                sp500_ticker_data.loc[symbols, 'volume'] = bar.v

        """
        ############################################################ 